# Capstone Project - Car Accident Severity

## 1 Introduction / Business Problem

### Stakeholders:
* Seattle Department of Transportation 
* Seattle Police Department - Traffic Enforcement Section
* Car Drivers in Seattle

Car accident is serious problem in Seattle. During the first six months of 2019, 101 people were seriously injured or killed in 98 collisions on Seattle streets, which is the highest number of crashes in the first half of a year since 2010, according to the data provided by the Seattle Department of Transportation (SDOT). The accident victims and their families, the insurance conpanies, health care personal and even the ordinary people, are affected by traffic accidents in many ways. Therefore, predicting the possibility and severity of a car accident based on the weather, road conditions and other factors is important. For drivers, they would drive more carefully or even change their travel if they are able to. For the police and traffic departments, they can put up some warning signs when there is a high traffic accident risk.

## 2. Data

The dataset used in this analysis is provided by SDOT Traffic Management Division, Traffic Records Group. It includes all types of collisions at the intersection or mid-block of a segment recorded by the Traffic Records from 2004 to Present.

The dataset contains 194,673 records with 38 attributes: 16 numerical and 22 categorical (see the figures below for the detailed descriptions). Among these 38 features, 6 of them have a large portion of null value, which are INTKEY, EXCEPTRSNCODE, EXCEPTRSNDESC, INATTENTIONIND, PEDROWNOTGRNT, SPEEDING. We will perform imputation on these feature during the feature engineering step. The target for this prediction task is the "Accident Severity", which are represented by three attributes: SEVERITYCODE, SEVERITYCODE.1, SEVERITYDESC, containing two sets of values: 1, 1, Property Damage Only Collision and 2, 2, Injury Collision. The target is unevenly distributed, with ~70% Property Damage Collisions Only and ~30% Injury Collisions. Therefore, target weights will be applied in the modeling step.

![](SDOT1.png)

![](SDOT2.png)

![](SDOT3.png)

In [5]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)

In [7]:
df = pd.read_csv('Data-Collisions.csv', low_memory=False)
df.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,LOCATION,EXCEPTRSNCODE,EXCEPTRSNDESC,SEVERITYCODE.1,SEVERITYDESC,COLLISIONTYPE,PERSONCOUNT,PEDCOUNT,PEDCYLCOUNT,VEHCOUNT,INCDATE,INCDTTM,JUNCTIONTYPE,SDOT_COLCODE,SDOT_COLDESC,INATTENTIONIND,UNDERINFL,WEATHER,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,5TH AVE NE AND NE 103RD ST,,NaN,2,Injury Collision,Angles,2,0,0,2,2013/03/27 00:00:00+00,3/27/2013 2:54:00 PM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Overcast,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,AURORA BR BETWEEN RAYE ST AND BRIDGE WAY N,NaN,NaN,1,Property Damage Only Collision,Sideswipe,2,0,0,2,2006/12/20 00:00:00+00,12/20/2006 6:55:00 PM,Mid-Block (not related to intersection),16,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, LEFT SIDE ...",NaN,0,Raining,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,4TH AVE BETWEEN SENECA ST AND UNIVERSITY ST,NaN,NaN,1,Property Damage Only Collision,Parked Car,4,0,0,3,2004/11/18 00:00:00+00,11/18/2004 10:20:00 AM,Mid-Block (not related to intersection),14,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, REAR END",NaN,0,Overcast,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,2ND AVE BETWEEN MARION ST AND MADISON ST,,NaN,1,Property Damage Only Collision,Other,3,0,0,3,2013/03/29 00:00:00+00,3/29/2013 9:26:00 AM,Mid-Block (not related to intersection),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,N,Clear,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,SWIFT AVE S AND SWIFT AV OFF RP,NaN,NaN,2,Injury Collision,Angles,2,0,0,2,2004/01/28 00:00:00+00,1/28/2004 8:04:00 AM,At Intersection (intersection related),11,"MOTOR VEHICLE STRUCK MOTOR VEHICLE, FRONT END ...",NaN,0,Raining,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [36]:
df.shape

(194673, 38)

In [17]:
df.filter(regex='SEVERITY').drop_duplicates()

,SEVERITYCODE,SEVERITYCODE.1,SEVERITYDESC
0,2,2,Injury Collision
1,1,1,Property Damage Only Collision


In [19]:
df['SEVERITYCODE'].value_counts(normalize=True)

1    0.701099
2    0.298901
Name: SEVERITYCODE, dtype: float64

In [23]:
df.dtypes.value_counts()

object     22
int64      12
float64     4
dtype: int64

In [42]:
df_na = df.isna().mean().to_frame()
df_na[df_na.iloc[:, 0] > 0.50]

,0
INTKEY,0.665747
EXCEPTRSNCODE,0.564341
EXCEPTRSNDESC,0.971039
INATTENTIONIND,0.846897
PEDROWNOTGRNT,0.976026
SPEEDING,0.952058
